In [ ]:
# import the required libraries
from czitools import pylibczirw_metadata as czimd
from czitools import pylibczirw_tools as czird
from czitools import misc, napari_tools
from ipyfilechooser import FileChooser
from IPython.display import display, HTML
import napari
from pathlib import Path
import os
import dask.array as da

In [ ]:
defaultdir = os.path.join(Path(os.getcwd()).resolve().parents[1], "data")
fc = FileChooser(defaultdir)
fc.filter_pattern = '*.czi'
display(fc)

In [ ]:
# Print the selected path, filename, or both
print(fc.selected_path)
print(fc.selected_filename)
print(fc.selected)

In [ ]:
# get the complete metadata at once as one big class
mdata = czimd.CziMetadata(fc.selected)

# get the CZI metadata dictionary directly from filename
mdict = czimd.create_mdict_red(mdata, sort=False)

# convert metadata dictionary to a pandas dataframe
mdframe = misc.md2dataframe(mdict)

# and display it nicely as a HTML inside the jupyter notebook
display(HTML(mdframe.to_html()))

In [ ]:
# return array with dimension order STZCYXA
array6d, mdata, dimstring = czird.read_6darray(fc.selected,
                                               use_dask=True,
                                               chunk_zyx=True,
                                               output_order="STCZYX")

# remove A dimension do display the array inside Napari
dim_order, dim_index, dim_valid = czimd.CziMetadata.get_dimorder(dimstring)

# show dask array structure
if isinstance(array6d, da.array):
    array6d

out = type(array6d)
print(out)

In [ ]:
# show array inside napari viewer
viewer = napari.Viewer()
layers = napari_tools.show(viewer, array6d, mdata,
                           dim_string=dimstring,
                           blending="additive",
                           contrast='napari_auto',
                           gamma=0.85,
                           add_mdtable=True,
                           name_sliders=True)

In [ ]:
napari.utils.nbscreenshot(viewer)